In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ssl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pingouin as pg

sns.set(rc={'figure.figsize':(20, 10)})
warnings.filterwarnings("ignore")
%matplotlib inline
SEED = 23

In [9]:
df = pd.read_csv('/Users/ira/Downloads/REALEC_Inspector/data/part_experiment_result.csv')

In [10]:
df.head()

,av_depth,max_depth,min_depth,num_acl,num_rel_cl,num_advcl,num_sent,num_tok,av_tok_before_root,av_len_sent,...,punct_mistakes_because,punct_mistakes_but,punct_mistakes_compare,million_mistake,side_mistake,name,text,class,type,part
0,4.111111,6,2,2,0,1,9,150,5.444444,16.666667,...,0,0,0,0,0,./data/exam/exam2017/OBy_146_1,This two line graphs illustrates monthly avera...,60,1,0
1,3.000000,4,2,0,0,0,9,94,3.333333,10.444444,...,0,0,0,0,0,./data/exam/exam2017/OBy_146_1,In another hand Rio de Janeiro warmer. Minimum...,60,1,1
2,4.750000,6,4,0,1,0,4,77,6.000000,19.250000,...,0,0,0,0,0,./data/exam/exam2017/EGe_15_1,"﻿We have two graphics, which show us the popul...",65,1,0
3,6.333333,10,4,1,2,1,3,98,6.666667,32.666667,...,0,0,0,0,0,./data/exam/exam2017/EGe_15_1,The second graph shows us that during all this...,65,1,1
4,3.777778,6,0,3,0,5,9,155,5.222222,17.222222,...,0,0,0,0,0,./data/exam/exam2017/DOv_2_2,More and more young people are stunding on the...,60,2,0


In [11]:
df['class'] = round(df['class'] / 10).astype(int)
df = df[df['class'] != 1]
df = df.drop(['name', 'text', 'side_mistake',
              'vs', 'squared_vs', 'ttr', 'root_ttr',
              'log_ttr', 'uber_ttr', 'd', 'vvi',
              'squared_vv', 'punct_mistakes_pp',
              'punct_mistakes_because', 'class',
              'punct_mistakes_but', 'type', 'punct_mistakes_compare'], axis=1)

df.head(2)

,av_depth,max_depth,min_depth,num_acl,num_rel_cl,num_advcl,num_sent,num_tok,av_tok_before_root,av_len_sent,...,num_pres_plur,num_past_part,num_past_simple,num_linkings,num_4grams,num_func_ngrams,num_shell_noun,num_misspelled_tokens,million_mistake,part
0,4.111111,6,2,2,0,1,9,150,5.444444,16.666667,...,0,2,0,4,0,0,0,2,0,0
1,3.000000,4,2,0,0,0,9,94,3.333333,10.444444,...,0,0,0,1,1,0,0,1,0,1


In [15]:
def get_info(feature, df):
    mean_1 = round(np.mean(df[df['part'] == 0][feature].to_list()), 3)
    mean_2 = round(np.mean(df[df['part'] == 1][feature].to_list()), 3)
    one_way_anova = pg.anova(data=df, dv=feature, between='part', detailed=True)
    p_value = one_way_anova['p-unc'][0]
    F = one_way_anova['F'][0]
    np2 = one_way_anova['np2'][0]
    return round(p_value, 3), round(F, 3), round(np2, 3), mean_1, mean_2

In [16]:
p_values = []
Fs = []
np2s = []
mean_1s = []
mean_2s = []
features = [x for x in df.columns if x != 'part']
for f in features:
    p_value, F, np2, mean_1, mean_2 = get_info(f, df)
    p_values.append(p_value)
    Fs.append(F)
    np2s.append(np2)
    mean_1s.append(mean_1)
    mean_2s.append(mean_2)
f_df_anova = pd.DataFrame(data={'F-value': Fs,
                                'p-value': p_values,
                                'partial eta-square': np2s,
                                'feature': features,
                                'part 1': mean_1s,
                               'part 2': mean_2s})
f_df_anova = f_df_anova.sort_values(['F-value', 'partial eta-square'], ascending=[0, 0])
f_df_anova = f_df_anova.set_index('feature')

In [17]:
f_df_anova

,F-value,p-value,partial eta-square,part 1 mean,part 2 mean
feature,,,,,
num_poss,831.209,0.000,0.108,8.612,6.190
num_sent,796.005,0.000,0.104,7.036,5.479
lv,646.380,0.000,0.086,0.559,0.604
num_tok,642.133,0.000,0.085,141.441,115.540
num_tu,534.080,0.000,0.072,8.810,6.841
ndw,421.474,0.000,0.058,77.939,67.454
num_pres_sing,402.215,0.000,0.055,4.299,3.128
mci,366.312,0.000,0.051,2.867,2.256
num_adj_noun,331.277,0.000,0.046,8.319,6.536
